In [20]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load from .env file

api_key = os.getenv("GROCKEY")

In [22]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0,
    groq_api_key=api_key
)

res = llm.invoke("the first person to land on mooon")
print(res.content)

You're referring to one of the most historic moments in space exploration!

The first person to land on the Moon was Neil Armstrong, an American astronaut and commander of the Apollo 11 mission. On July 20, 1969, Armstrong stepped out of the lunar module Eagle and onto the Moon's surface, famously declaring, "That's one small step for man, one giant leap for mankind."

Here's a brief rundown of the Apollo 11 mission and Neil Armstrong's achievement:

**Apollo 11 Mission:**

* Launched on July 16, 1969, from Kennedy Space Center, Florida
* Crew members: Neil Armstrong (Commander), Edwin "Buzz" Aldrin (Lunar Module Pilot), and Michael Collins (Command Module Pilot)
* Landed on the Moon's surface in the Sea of Tranquility on July 20, 1969

**Neil Armstrong's Moonwalk:**

* Armstrong became the first person to set foot on the Moon at 2:56 UTC on July 21, 1969 (July 20 in the United States)
* He spent about two and a half hours outside the lunar module, collecting samples and conducting exp

In [3]:
import chromadb
chroma_client = chromadb.Client()

In [7]:
collection = chroma_client.create_collection(name="email_agent")

In [8]:
collection.add(
    ids=["id1", "id2"],
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ]
)


/home/deepak/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|█| 79.


In [10]:
documents =  collection.get(ids=['id1'])

In [11]:
documents

{'ids': ['id1'],
 'embeddings': None,
 'documents': ['This is a document about pineapple'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None]}

In [14]:
results = collection.query(
    query_texts=["query is about nagpur"], # Chroma will embed this for you
    n_results=1 # how many results to return
)
print(results)


{'ids': [['id1']], 'embeddings': None, 'documents': [['This is a document about pineapple']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None]], 'distances': [[1.5936821699142456]]}


In [18]:
collection.add(
    ids=["id3", "id4"],
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    metadatas=[
        { "url": "https://en.wikipedia.org/wiki/Pinnacle"},
        {"url": "https://en.wikipedia.org/wiki/Orange"}
    ]
)


In [23]:
# scrape webpage
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.linkedin.com/jobs/view/4270424474/?alternateChannel=search&eBP=CwEAAAGYOq3aIFXuytXU-1BzwV9z3jpCTNoTw4OIEyAahQhV0w1GG1MFXbR_G3wEjmASjrMIwwn4he3qMI6-DSksAyqNZiVLINsvLEYOqC3eB-xBF-eYcqj3Veh9kuZ6SAF5bZ0MtrFUy0XAgivDGDhXqPKA4LtKZaH8X9PIKp21L0BwRycBJsvRipTmtPc3oOtsy5sVP4C9tcLz0NWjBKKeLkUMWJFQHtvfSN5OV-2rK1ayOWqmixUyculrAsCpqOSD7NOuXPY7uSjcqZ1Yp7mJYOMxCBvF-x_kchIx0XUYkANJfLB_48f1FUX1ev2oWmeqaCRQNkajSx7E9_rCQmPJQRTWmNoUKXw0Lw77SlGcmddnFLzrH4pusIGufca8c15kSZF8p_ExLOhEENpeCRopgU3y-r_1PJZOfM7LebqXawoToByoLESFEV1s_JaEBZZ_YUw44PzOhvThMAQlBvM_VndrQCetLgWPG2ZkufOfPp7CkIvRUjFN98yL4XinQNEtHdxrbGZ6ClP8&refId=4FQt2Osf9o4MkrzyJCfaBw%3D%3D&trackingId=oWpVh03V9FIEbBUua9ybXA%3D%3D")
page_data = loader.load().pop().page_content


In [28]:
import re

# Remove repeated newlines and excessive spaces
cleaned_text = re.sub(r'\s+', ' ', page_data).strip()

print(cleaned_text)

Zeller hiring Sr. Software Engineer (AWS) - Backend in India | LinkedIn Skip to main content LinkedIn Sr. Software Engineer (AWS) - Backend in Mumbai Expand search This button displays the currently selected search type. When expanded it provides a list of search options that will switch the search inputs to match the current selection. Jobs People Learning Clear text Clear text Clear text Clear text Clear text Join now Sign in Sr. Software Engineer (AWS) - Backend Zeller India Apply Join or sign in to find your next job Join to apply for the Sr. Software Engineer (AWS) - Backend role at Zeller Not you? Remove photo First name Last name Email Password (6+ characters) By clicking Agree & Join, you agree to the LinkedIn User Agreement, Privacy Policy and Cookie Policy. Continue Agree & Join or You may also apply directly on company website. Security verification Already on LinkedIn? Sign in Sr. Software Engineer (AWS) - Backend Zeller India 3 days ago 173 applicants See who Zeller has hi

In [41]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':cleaned_text})
type(res.content)

str

In [42]:
print(res, type(res))

content='```json\n{\n    "role": "Sr. Software Engineer (AWS) - Backend",\n    "experience": "5+ years of working experience in a technical hands-on software engineering role",\n    "skills": [\n        "Node.js",\n        "TypeScript",\n        "AWS",\n        "Microservices",\n        "Serverless architecture",\n        "CQRS",\n        "Event-sourcing",\n        "Jest",\n        "Supertest",\n        "Postman",\n        "NestJs",\n        "Multi-threaded programming",\n        "Socket programming",\n        "DynamoDB",\n        "SQL",\n        "DocumentDB",\n        "S3",\n        "SNS/SQS",\n        "Step Functions",\n        "Lambda",\n        "Cloud native architecture",\n        "Terraform",\n        "Cloudformation",\n        "CodePipeline",\n        "CodeBuild",\n        "CodeDeploy",\n        "CloudWatch",\n        "X-Ray"\n    ],\n    "description": "As a Senior Software Engineer, you will be responsible for architecting and developing cloud-native, highly available, robust,

In [43]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Sr. Software Engineer (AWS) - Backend',
 'experience': '5+ years of working experience in a technical hands-on software engineering role',
 'skills': ['Node.js',
  'TypeScript',
  'AWS',
  'Microservices',
  'Serverless architecture',
  'CQRS',
  'Event-sourcing',
  'Jest',
  'Supertest',
  'Postman',
  'NestJs',
  'Multi-threaded programming',
  'Socket programming',
  'DynamoDB',
  'SQL',
  'DocumentDB',
  'S3',
  'SNS/SQS',
  'Step Functions',
  'Lambda',
  'Cloud native architecture',
  'Terraform',
  'Cloudformation',
  'CodePipeline',
  'CodeBuild',
  'CodeDeploy',
  'CloudWatch',
  'X-Ray'],
 'description': 'As a Senior Software Engineer, you will be responsible for architecting and developing cloud-native, highly available, robust, and secure applications in the AWS environment. You will have experience in web, mobile, backend, API, database development, and should have experience in leading a team of software engineers.'}

In [45]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [46]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [49]:
links = collection.query(query_texts=json_res['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.co

In [50]:
json_res

{'role': 'Sr. Software Engineer (AWS) - Backend',
 'experience': '5+ years of working experience in a technical hands-on software engineering role',
 'skills': ['Node.js',
  'TypeScript',
  'AWS',
  'Microservices',
  'Serverless architecture',
  'CQRS',
  'Event-sourcing',
  'Jest',
  'Supertest',
  'Postman',
  'NestJs',
  'Multi-threaded programming',
  'Socket programming',
  'DynamoDB',
  'SQL',
  'DocumentDB',
  'S3',
  'SNS/SQS',
  'Step Functions',
  'Lambda',
  'Cloud native architecture',
  'Terraform',
  'Cloudformation',
  'CodePipeline',
  'CodeBuild',
  'CodeDeploy',
  'CloudWatch',
  'X-Ray'],
 'description': 'As a Senior Software Engineer, you will be responsible for architecting and developing cloud-native, highly available, robust, and secure applications in the AWS environment. You will have experience in web, mobile, backend, API, database development, and should have experience in leading a team of software engineers.'}

In [51]:
job = json_res
job['skills']

['Node.js',
 'TypeScript',
 'AWS',
 'Microservices',
 'Serverless architecture',
 'CQRS',
 'Event-sourcing',
 'Jest',
 'Supertest',
 'Postman',
 'NestJs',
 'Multi-threaded programming',
 'Socket programming',
 'DynamoDB',
 'SQL',
 'DocumentDB',
 'S3',
 'SNS/SQS',
 'Step Functions',
 'Lambda',
 'Cloud native architecture',
 'Terraform',
 'Cloudformation',
 'CodePipeline',
 'CodeBuild',
 'CodeDeploy',
 'CloudWatch',
 'X-Ray']

In [53]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Deepak, a business development executive at FlickO. FlickO is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of FlickO 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase FlickO's portfolio: {link_list}
        Remember you are Deepak, BDE at FlickO. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solution for Your Sr. Software Engineer (AWS) - Backend Needs

Dear Hiring Manager,

I came across the job description for a Sr. Software Engineer (AWS) - Backend at your organization, and I believe FlickO can be the perfect partner to fulfill your requirements. With a proven track record of delivering tailored software solutions, we can empower your enterprise with scalable, optimized, and highly efficient applications.

The job description highlights the need for expertise in Node.js, TypeScript, AWS, Microservices, Serverless architecture, and more. Our team at FlickO comprises seasoned professionals with hands-on experience in these technologies. We have successfully developed and deployed numerous cloud-native applications, leveraging the full potential of AWS services like DynamoDB, S3, SNS/SQS, Step Functions, and Lambda.

Some of our notable projects showcase our capabilities in full-stack JavaScript development, which aligns with your requirements. You can expl